In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
tf.random.set_seed(1234)

In [3]:
import pandas as pd

In [4]:
import glob

folder_path = "/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling"  # Replace with the actual path to your folder

# Use glob to get a list of all files in the folder
files = glob.glob(folder_path + "/*")

# Now, the 'files' variable contains a list of file paths in the specified folder
print(files)


['/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling13.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling5.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling12.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling16.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling14.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling6.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling4.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling11.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling2.csv', '/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main/falling/falling10.csv', '/content/drive/MyDrive/lidar_fall_detectio

In [5]:
def parse_dataset(num_points=400):

    train_points = []
    train_labels = []
    class_map = {}
    folder_path = "/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main"
    folders = glob.glob(folder_path + "/*")

    for i, folder in enumerate(folders):
        print("processing class: {}".format(os.path.basename(folder)))
        # store folder name with ID so we can retrieve later
        class_map[i] = folder.split("/")[-1]
        # gather all files

        train_files = glob.glob(os.path.join(folder_path,folder)+"/*")

        for f in train_files:


            # Use the random indices to select 200 random rows from the original array
            train_points.append(pd.read_csv(f).set_index('Unnamed: 0').sample(num_points).values)
            train_labels.append(i)

    return (
        np.array(train_points),
        np.array(train_labels),
        class_map,
    )

In [6]:
train_points = []
train_labels = []
class_map = {}
folder_path = "/content/drive/MyDrive/lidar_fall_detection/annotation/annotation_main"
folders = glob.glob(folder_path + "/*")

for i, folder in enumerate(folders):
  print("processing class: {}".format(os.path.basename(folder)))
  # store folder name with ID so we can retrieve later
  class_map[i] = folder.split("/")[-1]
  # gather all files

  train_files = glob.glob(os.path.join(folder_path,folder)+"/*")

  for f in train_files:
    train_points.append(pd.read_csv(f).set_index('Unnamed: 0'))
    train_labels.append(i)

processing class: falling
processing class: lying_down
processing class: sitting
processing class: standing


In [7]:
train_points[0].shape

(698, 3)

In [8]:
NUM_POINTS = 260
NUM_CLASSES = 4
BATCH_SIZE = 32
train_points,train_labels,class_map = parse_dataset(NUM_POINTS)

processing class: falling
processing class: lying_down
processing class: sitting
processing class: standing


In [9]:
train_points.shape

(296, 260, 3)

In [14]:
import sys
sys.path.append('/content/drive/MyDrive/lidar_fall_detection')
import tf_util

In [50]:
def get_model(point_cloud, is_training, bn_decay=None):
    """ Classification PointNet, input is BxNx3, output Bx40 """
    batch_size = point_cloud.get_shape()[0]
    num_point = point_cloud.get_shape()[1]
    end_points = {}
    input_image = tf.expand_dims(point_cloud, -1)

    # Point functions (MLP implemented as conv2d)
    net = tf_util.conv2d(input_image, 64, [1,3],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv1', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv2', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv3', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv4', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 1024, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv5', bn_decay=bn_decay)

    # Symmetric function: max pooling
    net = tf_util.max_pool2d(net, [num_point,1],
                             padding='VALID', scope='maxpool')

    # MLP on global point cloud vector
    net = tf.reshape(net, [batch_size, -1])
    net = tf_util.fully_connected(net, 512, bn=True, is_training=is_training,
                                  scope='fc1', bn_decay=bn_decay)
    net = tf_util.fully_connected(net, 256, bn=True, is_training=is_training,
                                  scope='fc2', bn_decay=bn_decay)
    net = tf_util.dropout(net, keep_prob=0.7, is_training=is_training,
                          scope='dp1')
    net = tf_util.fully_connected(net, 4, activation_fn=None, scope='fc3')

    return net, end_points


def get_loss(pred, label,end_points):
    """ pred: B*NUM_CLASSES,
        label: B, """
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred, labels=label)
    classify_loss = tf.reduce_mean(loss)
    tf.summary.scalar('classify loss', classify_loss)
    return classify_loss

In [57]:
BATCH_SIZE = 37

In [58]:
tf.compat.v1.disable_eager_execution()
point_cloud = tf.compat.v1.placeholder(tf.float32, shape=(BATCH_SIZE, NUM_POINTS, 3))
label = tf.compat.v1.placeholder(tf.int32, shape=(BATCH_SIZE,))
is_training = tf.constant(True)  # Set to True for training, False for inference


In [59]:
pred, end_points = get_model(point_cloud, is_training)

In [60]:
learning_rate = 0.001  # Adjust this as needed
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(get_loss(pred, label, end_points))

In [61]:
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [62]:
train_points.shape

(296, 260, 3)

In [ ]:
num_epochs=50
for epoch in range(num_epochs):
    # Loop through the dataset in mini-batches
    for i in range(0, len(train_points), BATCH_SIZE):
        batch_data = train_points[i:i+BATCH_SIZE]
        batch_labels = train_labels[i:i+BATCH_SIZE]

        #print("Batch data shape:", batch_data.shape)
        #print("Batch labels shape:", batch_labels.shape)

        # Feed batch data into the model and run the training operation
        feed_dict = {
            point_cloud: batch_data,
            label: batch_labels,
            is_training: True  # Set to True for training
        }
        _, loss_val = sess.run([train_op, get_loss(pred, label, end_points)], feed_dict=feed_dict)

        # Print loss for monitoring
    print(f"Epoch {epoch+1}/{num_epochs} , Loss: {loss_val:.4f}")

Epoch 1/50 , Loss: 2.1549
Epoch 2/50 , Loss: 2.3441
Epoch 3/50 , Loss: 2.6451
Epoch 4/50 , Loss: 1.6587
Epoch 5/50 , Loss: 1.3829
Epoch 6/50 , Loss: 1.5203
Epoch 7/50 , Loss: 1.1913
Epoch 8/50 , Loss: 1.1506
Epoch 9/50 , Loss: 1.5447
Epoch 10/50 , Loss: 0.8114
Epoch 11/50 , Loss: 0.9340
Epoch 12/50 , Loss: 0.8116
Epoch 13/50 , Loss: 0.6239
Epoch 14/50 , Loss: 0.6227
Epoch 15/50 , Loss: 0.5560
Epoch 16/50 , Loss: 0.4463
Epoch 17/50 , Loss: 0.7066
Epoch 18/50 , Loss: 0.4898
Epoch 19/50 , Loss: 0.7594
